In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append('../')

from typing import List,Dict
from elmo_on_md.data_loaders.sentiment_loader import SentimentLoader
from elmo_on_md.evaluation.sentiment_analysis import SentimentAnalysis
from elmo_on_md.evaluation.model_loader import load_model
from ELMoForManyLangs.elmoformanylangs import Embedder

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
loader = SentimentLoader()
sentiment_data = loader.load_data()

In [ ]:
elmo = load_model('original')
sentiment = SentimentAnalysis(elmo,lr=1e-4)
sentences = sentiment_data['train']['sentences']
labels = sentiment_data['train']['labels']

tokens_train,tokens_test, labels_train,labels_test = train_test_split(sentences, labels, test_size=0.2, random_state=1)

train_set = {'sentences':tokens_train,'labels':labels_train}
validate_set = {'sentences':tokens_test,'labels':labels_test}

sentiment.train(train_set,validate_set,n_epochs=40, batch_size = 32)

2019-08-17 21:59:53,973 INFO: char embedding size: 2289
2019-08-17 21:59:54,655 INFO: word embedding size: 189561
2019-08-17 21:59:59,341 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(189561, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2289, 50, padding_idx=2286)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_fe

0.0001


In [ ]:
train_preds = sentiment.predict(sentiment_data['train'])
print(confusion_matrix(sentiment_data['train']['labels'],train_preds))
test_preds = sentiment.predict(sentiment_data['test'])
print(confusion_matrix(sentiment_data['test']['labels'],test_preds))